In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.metrics import mean_squared_error, make_scorer
from scipy.stats import skew
from IPython.display import display

import pandas as pd
import numpy as np
import random
import os

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
train_df= pd.read_csv(r"C:\Users\USER\OneDrive - 한국외국어대학교\바탕 화면\2023 winter\전력사용량 ai\train.csv")
test_df= pd.read_csv(r"C:\Users\USER\OneDrive - 한국외국어대학교\바탕 화면\2023 winter\전력사용량 ai\test.csv")
building_df = pd.read_csv(r"C:\Users\USER\OneDrive - 한국외국어대학교\바탕 화면\2023 winter\전력사용량 ai\building_info.csv")
sample_df = pd.read_csv(r"C:\Users\USER\OneDrive - 한국외국어대학교\바탕 화면\2023 winter\전력사용량 ai\sample_submission.csv")

In [ ]:
#결측치
train_df['강수량(mm)'].fillna(0, inplace=True)
train_df['풍속(m/s)'].fillna(train_df['풍속(m/s)'].mean(), inplace=True)
train_df['습도(%)'].fillna(train_df['습도(%)'].mean(), inplace=True)
train_df['일조(hr)'].fillna(0, inplace=True)
train_df['일사(MJ/m2)'].fillna(0, inplace=True)
test_df['강수량(mm)'].fillna(0, inplace=True)
test_df['일조(hr)'] = 0
test_df['일사(MJ/m2)'] = 0

In [ ]:
# 날짜 및 시간 특성 파생
train_df['일시'] = pd.to_datetime(train_df['일시'], format='%Y%m%d %H')
test_df['일시'] = pd.to_datetime(test_df['일시'], format='%Y%m%d %H')
train_df['연'] = train_df['일시'].dt.year
train_df['월'] = train_df['일시'].dt.month
train_df['일'] = train_df['일시'].dt.day
train_df['시간'] = train_df['일시'].dt.hour
test_df['연'] = test_df['일시'].dt.year
test_df['월'] = test_df['일시'].dt.month
test_df['일'] = test_df['일시'].dt.day
test_df['시간'] = test_df['일시'].dt.hour

In [ ]:
# 필요하지 않은 컬럼 제거
train_df.drop(columns=['num_date_time', '일시'], inplace=True)
test_df.drop(columns=['num_date_time', '일시'], inplace=True)

In [ ]:
# 특성과 라벨 분리
X_train = train_df.drop(columns=['전력소비량(kWh)'])
y_train = train_df['전력소비량(kWh)']

In [ ]:
# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(test_df)

In [ ]:
# 데이터 분할 (훈련 및 검증 세트)
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)

In [ ]:
# 하이퍼파라미터 그리드 정의
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'n_estimators': [5000],
    'subsample': [0.7, 0.9],
    'colsample_bytree': [0.7, 0.9],
}

In [ ]:
# XGBoost 모델 생성
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)

In [ ]:
# 그리드 서치 객체 생성
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid,
                           cv=3, n_jobs=-1, verbose=2, scoring='neg_root_mean_squared_error')

In [ ]:
# 그리드 서치 수행
grid_search.fit(X_train_split, y_train_split, eval_set=[(X_val_split, y_val_split)], early_stopping_rounds=10, verbose=True)

In [ ]:
# 최적의 하이퍼파라미터 출력
print("Best Hyperparameters:", grid_search.best_params_)

In [ ]:
# 검증 세트에서 최적 모델의 성능 평가
y_val_pred_best = grid_search.best_estimator_.predict(X_val_split)
val_rmse_best = np.sqrt(mean_squared_error(y_val_split, y_val_pred_best))
print("Validation RMSE with Best Model:", val_rmse_best)

In [ ]:
# test 데이터에 대한 예측
y_test_pred_best = grid_search.best_estimator_.predict(X_test_scaled)
test_predictions_best = pd.DataFrame({'전력소비량(kWh)': y_test_pred_best})
test_predictions_best.to_csv('test_predictions_best.csv', index=False)